In [43]:
import pandas as pd

train = pd.read_csv("p2_data/data_train.csv", header=None)
target = train.shape[1]-1
y = train[target].to_numpy()
X = train.drop(columns=[target]).to_numpy()

test = pd.read_csv("p2_data/data_test.csv", header=None)
y_test = test[target]
X_test = test.drop(columns=[target])


In [68]:
from sklearn import tree
import random

class RandomForest:
    def __init__(self, n, depth):
        self.n = n
        self.depth = depth
        self.trees = []
        self.build()
        
    def build(self):
        for i in range(self.n):
            clf = tree.DecisionTreeClassifier(max_depth=self.depth)
            self.trees.append(clf)
    def fit(self, X, y, n):
        for i in range(self.n):
            features = []
            for j in range(n):
                features.append(random.randint(0, X.shape[1]-1))
            self.trees[i] = self.trees[i].fit(X[:, features], y)
#         print(X[0, features])
#         print(self.trees[len(self.trees)-1].predict(X[0, features].reshape(1, 3)))

In [69]:
clf = RandomForest(15, 3)
clf.fit(X, y, 3)

features
[2, 7, 6]
features
[11, 13, 1]
features
[11, 1, 8]
features
[6, 6, 11]
features
[0, 13, 12]
features
[13, 5, 15]
features
[10, 6, 3]
features
[4, 2, 9]
features
[14, 0, 8]
features
[11, 6, 8]
features
[3, 1, 14]
features
[10, 5, 4]
features
[7, 14, 14]
features
[0, 7, 3]
features
[6, 9, 2]
[26 23 27]
[2]
